In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df= pd.read_csv('/kaggle/input/bengaluru-house-price-data/Bengaluru_House_Data.csv')

In [ ]:
df.shape

In [ ]:
df.sample(5)

In [ ]:
df['location'].values

In [ ]:
df.duplicated().value_counts()

In [ ]:
df.drop_duplicates(keep='first',inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df['society'].value_counts()

In [ ]:
df.head()

In [ ]:
df['area_type'].value_counts()

In [ ]:
df['availability'].value_counts()

In [ ]:
df['location'].value_counts()

In [ ]:
df['size'].value_counts()

In [ ]:
df.corr()['price']

In [ ]:
df.dropna(subset=['size','location','bath','balcony'],inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
df['size']=df['size'].apply(lambda x:x.split()[0])

In [ ]:
df['size']=df['size'].astype('float')
df['bath']=df['bath'].astype('float')
df['balcony']=df['balcony'].astype('float')


In [ ]:
df.corr()['price']


In [ ]:
df.head()

In [ ]:
df['location'].unique()

In [ ]:
df['size'].unique()

In [ ]:
df['bath'].unique()


In [ ]:
df.drop(columns=['availability','society'],inplace=True)

In [ ]:
df.head()

In [ ]:
df['total_sqft']=df['total_sqft'].apply(lambda x:x.splitlines()[0])

In [ ]:
import re as re

In [ ]:
def find_number(text):
    num = re.findall(r'[0-9]+',text)
    return " ".join(num)
df['total_sqft']=df['total_sqft'].apply(lambda x: find_number(x))

In [ ]:
df['total_sqft']=df['total_sqft'].apply(lambda x:x.split()[0])

In [ ]:
df['total_sqft'].value_counts()

In [ ]:
df['total_sqft']=df['total_sqft'].astype('float')

In [ ]:
df.corr()['price']

In [ ]:
df['balcony'].value_counts()

In [ ]:
df['bath'].value_counts()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df['location']=df['location'].apply(lambda x: x.strip())
location_count=df['location'].value_counts()

In [ ]:
location_count_less_10= location_count[location_count<=10]
location_count_less_10

In [ ]:
df['location']=df['location'].apply(lambda x: 'other'if x in location_count_less_10 else x)

In [ ]:
df['location'].value_counts()

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
(df['total_sqft']/df['size']).describe()

In [ ]:
df=df[((df['total_sqft']/df['size'])>=300)]
df.describe()

In [ ]:
df=df[(df['total_sqft']<36000)]
df.describe()

In [ ]:
df['bath'].value_counts()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv("final_df.csv")

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
area_encoder= LabelEncoder()

In [ ]:
location_encoder=LabelEncoder()

In [ ]:

df['area_type']= area_encoder.fit_transform(df['area_type'])
df['location']= location_encoder.fit_transform(df['location'])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.corr()['price']

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
ohe= OneHotEncoder(drop='first')

In [ ]:
X=df.drop(columns=['area_type','location','price']).iloc[:,:].values

In [ ]:
X_trans=ohe.fit_transform(df[['area_type','location']]).toarray()

In [ ]:
X=np.hstack((X,X_trans))

In [ ]:
X

In [ ]:
X.shape

In [ ]:
y=df['price'].values

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2,random_state=0)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr= LinearRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
y_pred=lr.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,y_pred)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dct=DecisionTreeRegressor()

In [ ]:
dct.fit(X_train,y_train)

In [ ]:
y_pred1=dct.predict(X_test)

In [ ]:
r2_score(y_test,y_pred1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr=RandomForestRegressor(n_estimators=200)

In [ ]:
rfr.fit(X_train,y_train)

In [ ]:
y_pred2=rfr.predict(X_test)

In [ ]:
r2_score(y_test,y_pred2)

In [ ]:
from sklearn.linear_model import LogisticRegression 

In [ ]:
import pickle

In [ ]:
pickle.dump(location_encoder, open('location_encoder.pkl','wb'))


In [ ]:
pickle.dump(area_encoder, open('area_encoder.pkl','wb'))

In [ ]:
pickle.dump(ohe, open('ohe.pkl','wb'))

In [ ]:
pickle.dump(lr,open('model.pkl','wb'))

In [ ]:
pickle.dump(rfr,open('Rfrmodel.pkl','wb'))